In [1]:
from transformers import BertForQuestionAnswering, AutoTokenizer, pipeline

# Open the file and read its contents
# file_path = '/content/data.txt'
# with open(file_path, "r") as file:
#     data = file.read()

# Small data sample to test faster
data = "Neymar made his professional debut with Santos in 2009, and in 2011, he helped them win their first Copa Libertadores in nearly 50 years.[5] In 2013, he joined Barcelona and became part of an attacking trio with Lionel Messi and Luis Suárez, dubbed MSN. Winning the continental treble of La Liga, the Copa del Rey, and the UEFA Champions League in the trio's first season, Neymar was the joint-top scorer of the Champions League campaign and top scorer in the Copa del Rey. Neymar joined Paris Saint-Germain (PSG) in 2017 in a transfer costing €222 million, making him the most expensive player ever.[note 1][8] There, he won Ligue 1 Player of the Year, won five Ligue 1 titles, and was integral to PSG being runners-up in the 2019-20 Champions League. He also ranks as PSG's fourth-highest all-time top goalscorer, despite reoccurring injuries consistently disrupting his playing time. In 2023, he became the most expensive signing in Saudi Pro League history, costing €90 million, as he signed for Al Hilal. Debuting for Brazil aged 18, Neymar is the nation's all-time top goalscorer, with 79 goals in 128 matches. He won the 2013 FIFA Confederations Cup, winning the Golden Ball. In the 2014 FIFA World Cup, he was named in the Dream Team. He captained Brazil to their first Olympic gold medal in men's football at the 2016 Summer Olympics, having already achieving a silver medal at the 2012 edition. Helping Brazil to a runner-up finish at the 2021 Copa América, he was jointly awarded Best Player. In the 2022 World Cup, he became the third Brazilian player to score in three World Cups, after Pelé and Ronaldo. Neymar has won a record six Samba Gold awards. Neymar has been named in the FIFA FIFPro World11 and the UEFA Team of the Year twice and the UEFA Champions League Squad of the Season three times. He finished third for the FIFA Ballon d'Or in 2015 and 2017 and won the FIFA Puskás Award in 2011. SportsPro named Neymar the world's most marketable athlete in 2012 and 2013, and ESPN cited him as the world's fourth-most-famous athlete in 2016. In 2017, Time included him in its annual list of the 100 most influential people in the world.[9] France Football ranked Neymar the world's third-highest-paid footballer of 2018. Forbes ranked him the world's third-highest-paid athlete of 2019,[10] dropping to fourth in 2020.[11]"

# Define your questions
questions = [
    "Did Neymar play for Barcelona?",
    "Did Neymar play for Real Madrid?",
    "How many Champions Leagues has Neymar won?",
    "Where is Neymar from?",
    "Which players did Neymar play with?",
    "Has Neymar won a world cup?",
    "Is Neymar the best player in the world?"
]

# Load the pre-trained BERT model for question answering
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('deepset/bert-base-cased-squad2')

# Initialize the question answering pipeline
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

# Iterate over each question and provide answers from the file context
for question in questions:
    answer = nlp({
        'question': question,
        'context': data
    })
    print(f"Question: {question}")
    print(f"Answer: {answer['answer']}\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Question: Did Neymar play for Barcelona?
Answer: he joined Barcelona

Question: Did Neymar play for Real Madrid?
Answer: 2017

Question: How many Champions Leagues has Neymar won?
Answer: six

Question: Where is Neymar from?
Answer: Brazil

Question: Which players did Neymar play with?
Answer: Lionel Messi and Luis Suárez

Question: Has Neymar won a world cup?
Answer: 2022 World Cup,

Question: Is Neymar the best player in the world?
Answer: he was jointly awarded Best Player



Based on what I know as of this moment, I am seeing very different results depending on the data that I provide. When I use the data I scrape from my chatbot project I get worse answers, however when I use this small paragraph from wikipedia, I get more accurate answers. I also experiemented with my line of quesetioning. For example: "Did Neymar play for Real Madrid?" and "Did Neymar play for the Real Madrid team from Spain?" and got the same result. Out of the 7 questions, all were answered in the right format. So if the question was looking for a date, a date was provided. If a quantity was asked for, it was provided. The scraped data provided much worse asnwers, as the question and answers made no sense. When a quantiy was asked for, a region was provided. Some of the answers are incorrect, as when the text does not explicitly have the answer for a question, it provides soemthing close, which means providing the wrong answer. Maybe to train I can focus on answers where context is not as straight forward, and see if I can help it understand when there is no answer.

In [9]:
!pip install transformers[torch]
!pip install datasets
!pip install accelerate -U

In [3]:
from transformers import BertForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import numpy as np


# Load the pre-trained BERT model and ensure the tokenizer is a fast tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Define the data
data_list = [
    {
        "question": "Did Neymar play for Barcelona?",
        "context": "Neymar made his professional debut with Santos in 2009, and in 2011, he helped them win their first Copa Libertadores title since 1963. Neymar transferred to Barcelona in 2013, and won two La Liga titles, three Copa del Rey, and the UEFA Champions League in 2015.",
        "answers": {
            "text": ["Neymar transferred to Barcelona in 2013"],
            "answer_start": [98]
        }
    },
    {
        "question": "Where is Neymar from?",
        "context": "Debuting for Brazil aged 18, Neymar is the nation's all-time top goalscorer, with 79 goals in 128 matches.",
        "answers": {
            "text": ["Brazil"],
            "answer_start": [11]
        }
    },
    {
        "question": "Has Neymar won a world cup?",
        "context": "Helping Brazil to a runner-up finish at the 2021 Copa América, he was jointly awarded Best Player. In the 2022 World Cup, he became the third Brazilian player to score in three World Cups, after Pelé and Ronaldo. Neymar has won a record six Samba Gold awards.",
        "answers": {
            "text": ["No"],
            "answer_start": [197]
        }
    },
    {
        "question": "Has Neymar won the Copa Libertadores?",
        "context": "Neymar made his professional debut with Santos in 2009, and in 2011, he helped them win their first Copa Libertadores in nearly 50 years.",
        "answers": {
            "text": ["Yes"],
            "answer_start": [93]
        }
    },
    {
        "question": "When did Neymar join PSG?",
        "context": "Neymar joined Paris Saint-Germain (PSG) in 2017 in a transfer costing €222 million, making him the most expensive player ever.",
        "answers": {
            "text": ["2017"],
            "answer_start": [33]
        }
    },
    {
        "question": "Has Neymar won the league in France?",
        "context": "Neymar joined Paris Saint-Germain (PSG) in 2017 in a transfer costing €222 million, making him the most expensive player ever. There, he won Ligue 1 Player of the Year, won five Ligue 1 titles, and was integral to PSG being runners-up in the 2019-20 Champions League.",
        "answers": {
            "text": ["Yes"],
            "answer_start": [130]
        }
    },
    {
        "question": "Does Neymar get injured frequently?",
        "context": "He also ranks as PSG's fourth-highest all-time top goalscorer, despite reoccurring injuries consistently disrupting his playing time.",
        "answers": {
            "text": ["Yes"],
            "answer_start": [83]
        }
    },
    {
        "question": "Is Neymar influential?",
        "context": "In 2017, Time included him in its annual list of the 100 most influential people in the world.",
        "answers": {
            "text": ["Yes"],
            "answer_start": [9]
        }
    },
    {
        "question": "What club did Messi leave in 2021?",
        "context": "In August 2021, Lionel Messi left FC Barcelona after over two decades at the club, due to financial and structural obstacles.",
        "answers": {
            "text": ["FC Barcelona"],
            "answer_start": [35]
        }
    },
    {
        "question": "Where did Cristiano Ronaldo move in 2021?",
        "context": "Cristiano Ronaldo rejoined Manchester United in 2021 after leaving Juventus, marking a significant return to the English club where he had previously made his name.",
        "answers": {
            "text": ["Manchester United"],
            "answer_start": [27]
        }
    },
]

# Convert the data into a format suitable for the dataset
full_dataset = Dataset.from_dict({
    "question": [item["question"] for item in data_list],
    "context": [item["context"] for item in data_list],
    "answers": [item["answers"] for item in data_list]
})

# Split the dataset into training and validation sets
dataset = full_dataset.train_test_split(test_size=0.1)  # 10% for validation
dataset = DatasetDict({
    'train': dataset['train'],
    'test': dataset['test']
})

def prepare_train_features(examples):
    tokenized_inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt"
    )

    start_positions = []
    end_positions = []

    for i in range(len(examples["question"])):
        context = examples["context"][i]
        answer = examples["answers"][i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        offset_mapping = tokenized_inputs["offset_mapping"][i]
        sequence_ids = tokenized_inputs.sequence_ids(i)

        start_token_index = None
        end_token_index = None

        for j, (offset, seq_id) in enumerate(zip(offset_mapping, sequence_ids)):
            if seq_id == 1 and offset[0] <= start_char and offset[1] > start_char:
                start_token_index = j
            if seq_id == 1 and offset[0] < end_char and offset[1] >= end_char:
                end_token_index = j
                break

        if start_token_index is None or end_token_index is None:
            start_token_index = tokenizer.model_max_length
            end_token_index = tokenizer.model_max_length

        start_positions.append(start_token_index)
        end_positions.append(end_token_index)

    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions
    del tokenized_inputs["offset_mapping"]
    return tokenized_inputs

# Tokenize and align the input data
tokenized_dataset = dataset.map(prepare_train_features, batched=True)

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

# Define training arguments with evaluation strategy
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,  # Number of epochs
    per_device_train_batch_size=4,  # Batch size
    warmup_steps=500,  # Number of warmup steps
    weight_decay=0.01,  # Weight decay
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every 10 steps
    evaluation_strategy="steps",  # Evaluate every logging step
    eval_steps=50,  # Evaluation and logging every 50 steps
    save_strategy="steps",  # Save the model state every logging step
    save_steps=50,  # Save every 50 steps
    load_best_model_at_end=True  # Load the best model at the end of training
)

# Initialize the trainer with the train and validation datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

model.save_pretrained('./my_fine_tuned_model')
tokenizer.save_pretrained('./my_fine_tuned_model')


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss


('./my_fine_tuned_model/tokenizer_config.json',
 './my_fine_tuned_model/special_tokens_map.json',
 './my_fine_tuned_model/vocab.txt',
 './my_fine_tuned_model/added_tokens.json',
 './my_fine_tuned_model/tokenizer.json')

In [4]:
from transformers import BertForQuestionAnswering, AutoTokenizer
from transformers import pipeline

# Small data sample to test faster
data = "Neymar made his professional debut with Santos in 2009, and in 2011, he helped them win their first Copa Libertadores in nearly 50 years.[5] In 2013, he joined Barcelona and became part of an attacking trio with Lionel Messi and Luis Suárez, dubbed MSN. Winning the continental treble of La Liga, the Copa del Rey, and the UEFA Champions League in the trio's first season, Neymar was the joint-top scorer of the Champions League campaign and top scorer in the Copa del Rey. Neymar joined Paris Saint-Germain (PSG) in 2017 in a transfer costing €222 million, making him the most expensive player ever.[note 1][8] There, he won Ligue 1 Player of the Year, won five Ligue 1 titles, and was integral to PSG being runners-up in the 2019-20 Champions League. He also ranks as PSG's fourth-highest all-time top goalscorer, despite reoccurring injuries consistently disrupting his playing time. In 2023, he became the most expensive signing in Saudi Pro League history, costing €90 million, as he signed for Al Hilal. Debuting for Brazil aged 18, Neymar is the nation's all-time top goalscorer, with 79 goals in 128 matches. He won the 2013 FIFA Confederations Cup, winning the Golden Ball. In the 2014 FIFA World Cup, he was named in the Dream Team. He captained Brazil to their first Olympic gold medal in men's football at the 2016 Summer Olympics, having already achieving a silver medal at the 2012 edition. Helping Brazil to a runner-up finish at the 2021 Copa América, he was jointly awarded Best Player. In the 2022 World Cup, he became the third Brazilian player to score in three World Cups, after Pelé and Ronaldo. Neymar has won a record six Samba Gold awards. Neymar has been named in the FIFA FIFPro World11 and the UEFA Team of the Year twice and the UEFA Champions League Squad of the Season three times. He finished third for the FIFA Ballon d'Or in 2015 and 2017 and won the FIFA Puskás Award in 2011. SportsPro named Neymar the world's most marketable athlete in 2012 and 2013, and ESPN cited him as the world's fourth-most-famous athlete in 2016. In 2017, Time included him in its annual list of the 100 most influential people in the world.[9] France Football ranked Neymar the world's third-highest-paid footballer of 2018. Forbes ranked him the world's third-highest-paid athlete of 2019,[10] dropping to fourth in 2020.[11]"

questions = [
    "Did Neymar play for Barcelona?",
    "Did Neymar play for Real Madrid?",
    "How many Champions Leagues has Neymar won?",
    "Where is Neymar from?",
    "Which players did Neymar play with?",
    "Has Neymar won a world cup?",
    "Is Neymar the best player in the world?"
]

model_path = './my_fine_tuned_model'
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Iterate over each question and provide answers from the file context
for question in questions:
    answer = nlp({
        'question': question,
        'context': data
    })
    print(f"Question: {question}")
    print(f"Answer: {answer['answer']}\n")

Question: Did Neymar play for Barcelona?
Answer: in the world.[9] France Football ranked Neymar

Question: Did Neymar play for Real Madrid?
Answer: in the world.[9] France Football ranked Neymar

Question: How many Champions Leagues has Neymar won?
Answer: in three World Cups, after Pelé and Ronaldo. Neymar

Question: Where is Neymar from?
Answer: in the world.[9] France Football ranked Neymar

Question: Which players did Neymar play with?
Answer: in the world.[9] France Football ranked Neymar

Question: Has Neymar won a world cup?
Answer: in the world.[9] France Football ranked Neymar

Question: Is Neymar the best player in the world?
Answer: in the world.[9] France Football ranked Neymar



The fine tuned model is peforming incredibly poorly, I changed the block of code to fit the previous test (of the model before fine tuning) to make sure the process was the exact same. Loss function of 6 is very poor, so I think what I ned to do is figure out a way to create more examples for the model to train. Epochs were reduced, and 3 more training examples were added, model performs the same.